In [17]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import label_binarize

irisdataset = datasets.load_iris()
iris = np.column_stack([irisdataset['data'], irisdataset['target']])
np.random.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

learning_rate = 0.001
training_epochs = 30
batch_size = 10
display_step = 1

n_hidden_1 = 10
n_hidden_2 = 10
n_input = 4
n_classes = 3

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

logits = multilayer_perceptron(X)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size, :4], target_multiclass[i:i+batch_size]
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    pred = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=16.866773415
Epoch: 0002 cost=12.046640460
Epoch: 0003 cost=9.902506574
Epoch: 0004 cost=8.508365186
Epoch: 0005 cost=6.989565309
Epoch: 0006 cost=5.871528308
Epoch: 0007 cost=4.991677348
Epoch: 0008 cost=4.239563862
Epoch: 0009 cost=3.452436654
Epoch: 0010 cost=2.667988888
Epoch: 0011 cost=1.926836904
Epoch: 0012 cost=1.272819157
Epoch: 0013 cost=0.791796269
Epoch: 0014 cost=0.501855510
Epoch: 0015 cost=0.348417119
Epoch: 0016 cost=0.266970824
Epoch: 0017 cost=0.216989655
Epoch: 0018 cost=0.185341869
Epoch: 0019 cost=0.161990704
Epoch: 0020 cost=0.145044956
Epoch: 0021 cost=0.131662574
Epoch: 0022 cost=0.121029534
Epoch: 0023 cost=0.112326616
Epoch: 0024 cost=0.105025798
Epoch: 0025 cost=0.098834903
Epoch: 0026 cost=0.093475956
Epoch: 0027 cost=0.088784904
Epoch: 0028 cost=0.084627180
Epoch: 0029 cost=0.080901507
Epoch: 0030 cost=0.077533552
Optimization Finished!
Accuracy: 0.94
